# 2.3 - Learning to Match via Inverse Optimal Transport


The goal of this notebook is to implement [this paper](https://arxiv.org/abs/1909.04239), listed as paper 3 in our summary, with the movie lens dataset

In [ ]:
import numpy as np
import scipy.optimize
import matplotlib.pyplot as plt
from scipy.sparse import random as sparserandom
from sklearn.decomposition import PCA

# Pre-processing


Hyperparameters of the model for regularization, kernel function, and max iterations

In [ ]:
# HYPER PARAMETERS
n = 10 #Number of users
m = 12 #Number of movies

# Regularization parameters
lambda0 = 1
lambda_u = 1
lambda_v = 1
delta = 0.02

# Rank of sub space of users and movies and A
features_users = 9
features_movies = 8
rankA = 3

# Learning rate
s = 100

# nombre d'iterations du RIOT
L = 1000
K = 30

# Paramètres du kernel
gamma = 0.5
const = 1
deg = 2

Generate transport plan and its 2 marginals, as well as similarity matrices between users and movies

In [ ]:
def data_generator(n,m):
    Pi = sparserandom(n, m, density=0.4, data_rvs=np.ones).A
    N = np.sum(Pi)
    Pi /= N
    users = np.sum(Pi, axis=1)
    movies = np.sum(Pi, axis=0)
    return Pi, users, movies

# Generate Cu and Cv
def distance_matrix(vector):
    n = len(vector)
    vector = vector.reshape(n,1)
    return ((vector[:,None,:] - vector[None,:,:])**2).sum(2)**0.5

Useful functions for the RIOT algorithm

In [ ]:
def KL(a, b) :
    return (a * np.log(a / b)).sum()

def sinkhorn(C, users, movies, gamma =1, max_iter=100, threshold = 1e-6):
    n, m  = C.shape

    K = np.exp(-lambda0 * C)

    a, b = np.ones(n), np.ones(m)

    iteration, error = 0, np.inf

    while iteration < max_iter and error > threshold:
        next_b = movies / np.dot(K.T, a)
        next_a = users / np.dot(K, next_b)
        error = np.linalg.norm(next_a - a) + np.linalg.norm(next_b - b)
        a, b = next_a, next_b
    
    pi = np.dot(np.diag(a),  K * b)
    
    return pi, a, b

# To compute during the algorithm. We use a polynomial kernel
def C_of_A(U, V, A):
    return gamma * (U @ A @ V.T + const) ** deg

def dC_of_A(U, V, A):
    return deg * gamma * (U @ A @ V.T + const) **(deg-1)

# Find the root of the polynomial equation
def find_root_theta(marginal, Z, M, eta):
    
    def p(theta):
        return (marginal  / (M - theta * Z).dot(eta)).dot(Z).dot(eta)
    theta0 = np.min(M.dot(eta) / Z.dot(eta))
    return scipy.optimize.root(p, theta0).x[0]


# Algorithm run
We generate all the necessary data


In [ ]:
# Generate Data
Pi_hat, users, movies = data_generator(n, m)

Cu = distance_matrix(users)
Cv = distance_matrix(movies)

A = np.random.rand(features_users, features_movies)

# Projection of data on the latent space
pca_u = PCA(n_components=features_users)
pca_u.fit(Pi_hat.T)
U = pca_u.components_.T

pca_m = PCA(n_components=features_movies)
pca_m.fit(Pi_hat)
V = pca_m.components_.T

Initialization of the matrices

In [ ]:
## Initialize
C = C_of_A(U, V, A)

print(C)
z = (1/lambda_u) * np.log(sinkhorn(Cu, users, users, lambda_u)[2])
w = (1/lambda_v) * np.log(sinkhorn(Cv, movies, movies, lambda_v)[2])

Pi, ksi, eta = sinkhorn(C, users, movies)
ksi = np.exp(ksi * lambda0)
eta = np.exp(eta * lambda0)
print(ksi, eta)

Optimization loop

In [ ]:
## Looping optimization
for _ in range(L):
    
    Z = np.exp(- lambda0 * C)
    M = delta * (np.outer(z, np.ones(m)) + np.outer(np.ones(n), w)) * Z 
    for _ in range(K):

        theta_1 = find_root_theta(users, Z, M, eta)
        theta_2 = find_root_theta(movies, Z.T, M.T, ksi)
        ksi = users / ((M - theta_1 * Z) @ eta)
        eta = movies / ((M - theta_2 * Z).T @ ksi)
    
    ## the same as using a and b notations
    Pi = np.dot(np.diag(ksi), np.exp(-lambda0 * C) * eta)
    
    factor = lambda0 * (Pi_hat + (theta_1 - delta*(np.outer(z, np.ones(m)) + np.outer(np.ones(n), w))) * Pi) * dC_of_A(U, V, A)
    A -= s * U.T.dot(factor).dot(V)
    C = C_of_A(U, V, A)
    
    z = (1/lambda_u) * np.log(sinkhorn(Cu, np.sum(Pi, axis=1), users, lambda_u)[2])
    w = (1/lambda_v) * np.log(sinkhorn(Cv, np.sum(Pi, axis=0), movies, lambda_v)[2])


Result transport plan computed, and its cost matrice

In [ ]:
Pi, C